<a href="https://colab.research.google.com/github/15muralidhara/oxcourse/blob/main/Greyscale_to_color_converter_Hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using the CIFAR10 data provided below, create an alogrithm that takes in a gray scale image and coverts it to a color image.

Note for this problem we will define the grayscale image as the average of the three color channels.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

In [ ]:
train_data = CIFAR10(root='./datasets', train=True, download=True, transform=transforms.ToTensor())
test_data = CIFAR10(root='./datasets', train=True, download=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_data,batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_data,batch_size=64, shuffle=False, pin_memory=True)

# creating a GAN-based model for colorization
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Define architecture of the generator
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        # Initialize the biases with zeros
        for layer in self.model:
            if isinstance(layer, nn.Conv2d):
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=0)
        )
    def forward(self, x):
        return self.model(x)

# need to pool it to produce single scale not 2d
# for initialization of generator,

Files already downloaded and verified


In [1]:
#train model using binary cross-entropy loss and Adam optimizer

def train_colorization_gan(generator, discriminator, criterion, gen_optimizer, disc_optimizer, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    discriminator.to(device)

    for epoch in range(num_epochs):
        for i, (greyscale_images, color_images) in enumerate(train_loader):
            greyscale_images, color_images = greyscale_images.to(device).float(), color_images.to(device).float()
            real_labels = torch.ones(color_images.size(0), 1).to(device)
            fake_labels = torch.zeros(color_images.size(0), 1).to(device)

            color_images = color_images.repeat(1, 3, 1, 1)

            # Train the discriminator
            disc_optimizer.zero_grad()
            real_outputs = discriminator(color_images)
            real_loss = criterion(real_outputs, real_labels)

            fake_images = generator(greyscale_images)
            fake_outputs = discriminator(fake_images.detach())
            fake_loss = criterion(fake_outputs, fake_labels)


            disc_loss = (real_loss + fake_loss) / 2.0
            disc_loss.backward()
            disc_optimizer.step()

            # Train the generator
            gen_optimizer.zero_grad()
            fake_outputs = discriminator(fake_images)
            gen_loss = criterion(fake_outputs, real_labels)
            gen_loss.backward()
            gen_optimizer.step()

            if i % 100 == 0:
                print(f"Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(train_loader)}] Gen Loss: {gen_loss.item()} Disc Loss: {disc_loss.item()}")


# Instantiate the generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Define the loss and optimizers
criterion = nn.BCEWithLogitsLoss()
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
disc_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Train the GAN
num_epochs = 10
train_colorization_gan(generator, discriminator, criterion, gen_optimizer, disc_optimizer, num_epochs)

NameError: ignored

In [ ]:
# colourize

def colorize_images(generator, test_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator.to(device)
    generator.eval()

    colorized_images = []
    with torch.no_grad():
        for greyscale_images, _ in test_loader:
            greyscale_images = greyscale_images.to(device)
            color_images = generator(greyscale_images)
            colorized_images.append(color_images.cpu())

    return torch.cat(colorized_images, dim=0)


# Colorize test images
colorized_images = colorize_images(generator, test_loader)